In [ ]:
# %pip install 

In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
from faimed3d.all import *
from torchvision.models.video import r3d_18
from fastai.callback.all import SaveModelCallback
from torch import nn
import SimpleITK as sitk

In [3]:
REPO_DIR = Path(os.getcwd()).parent
DATA_DIR = Path('/Users/abhinavkbij/Documents/projects/pytorchIntro/data/lits17')

In [4]:
ct_images = list(DATA_DIR.glob("volume-*.nii"))
# ct_images.sort()
liver_tumor_mask = list(DATA_DIR.glob("segmentation-*.nii"))
# infect_mask.sort()

In [5]:
d = pd.DataFrame({'images': ct_images, 
                 'masks': liver_tumor_mask})

In [6]:
TensorDicom3D.create(d.images[0]).show()

In [7]:
TensorDicom3D.create(d.images[0]).show()

/opt/miniconda3/envs/faimed3d_py310/lib/python3.10/site-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)
/opt/miniconda3/envs/faimed3d_py310/lib/python3.10/site-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)


In [9]:
dls[0]

In [12]:
dls.one_batch()[1].shape

torch.Size([1, 1, 20, 224, 224])

In [ ]:
def get_sub_volume(image, label, patch_size=(20, 224, 224), max_tries=1000, bgr_threshold=0.95, num_patches=10):
    X, y = None, None
    patch_list = []
    orig_x, orig_y, orig_z = image.shape
    patch_x, patch_y, patch_z = patch_size
    
    tumor_label = label == 2
    tries = 0
    while tries<=max_tries:
        tries+=1

        start_x = np.random.randint(0, orig_x-patch_x)
        start_y = np.random.randint(0, orig_y-patch_y)
        start_z = np.random.randint(0, orig_z-patch_z)

        label_out = tumor_label[start_x:start_x+patch_x, start_y:start_y+patch_y, start_z:start_z+patch_z]

        bgr_ratio = np.sum(label_out==0)/np.prod(patch_size)
        if bgr_ratio<bgr_threshold:
            X = image[start_x:start_x+patch_x, start_y:start_y+patch_y, start_z:start_z+patch_z].copy()
            y = label_out.copy()
            if not y.shape == patch_size:
                continue
            if len(patch_list)<num_patches:
                patch_list.append((X, y))
                continue

            return patch_list
        else:
            continue
    return patch_list

In [85]:
def read_image(path):
    return sitk.ReadImage(path)

def read_image_mask_pair_as_numpy(row):
    return sitk.GetArrayFromImage(sitk.ReadImage(row.images)), sitk.GetArrayFromImage(sitk.ReadImage(row.masks))

def write_image_mask_pair_as_nii(patch_list, ref_image, image_path, label_path):
    path_list = []
    for idx, (image, label) in enumerate(patch_list):
        image = sitk.GetImageFromArray(image)
        label = sitk.GetImageFromArray(label.astype(np.int8))
        # print(image.shape, label.shape)
        # continue
        _, _, _ = image.SetSpacing(ref_image.GetSpacing()), image.SetDirection(ref_image.GetDirection()), image.SetOrigin(ref_image.GetOrigin())
        _, _, _ = label.SetSpacing(ref_image.GetSpacing()), label.SetDirection(ref_image.GetDirection()), label.SetOrigin(ref_image.GetOrigin()) 

        patch_img_path = image_path.parent / f"{image_path.stem}-patch-{idx}.nii"
        sitk.WriteImage(image, patch_img_path)
        patch_label_path = label_path.parent / f"{label_path.stem}-patch-{idx}.nii"
        sitk.WriteImage(label, patch_label_path)
        path_list.append((patch_img_path, patch_label_path))
    return path_list

In [88]:
# patch_d = d.copy()
patch_dl = []
for row in d.itertuples():
    image, label = read_image_mask_pair_as_numpy(row)
    print(type(image))
    patch_list = get_sub_volume(image, label)
    print(len(patch_list))

    # add to list
    # patch_dl.extend(patch_list)
    ref_image = sitk.ReadImage(row.images)
    path_list = write_image_mask_pair_as_nii(patch_list, ref_image, row.images, row.masks)
    # break
    patch_dl.extend(path_list)

patch_d = pd.DataFrame(columns=["images", "masks"], data=patch_dl)

<class 'numpy.ndarray'>
10
<class 'numpy.ndarray'>
10
<class 'numpy.ndarray'>
10
<class 'numpy.ndarray'>
10
<class 'numpy.ndarray'>
10
<class 'numpy.ndarray'>
10


In [89]:
patch_d.head()

,images,masks
0,/Users/abhinavkbij/Documents/projects/pytorchIntro/data/lits17/volume-4-patch-0.nii,/Users/abhinavkbij/Documents/projects/pytorchIntro/data/lits17/segmentation-5-patch-0.nii
1,/Users/abhinavkbij/Documents/projects/pytorchIntro/data/lits17/volume-4-patch-1.nii,/Users/abhinavkbij/Documents/projects/pytorchIntro/data/lits17/segmentation-5-patch-1.nii
2,/Users/abhinavkbij/Documents/projects/pytorchIntro/data/lits17/volume-4-patch-2.nii,/Users/abhinavkbij/Documents/projects/pytorchIntro/data/lits17/segmentation-5-patch-2.nii
3,/Users/abhinavkbij/Documents/projects/pytorchIntro/data/lits17/volume-4-patch-3.nii,/Users/abhinavkbij/Documents/projects/pytorchIntro/data/lits17/segmentation-5-patch-3.nii
4,/Users/abhinavkbij/Documents/projects/pytorchIntro/data/lits17/volume-4-patch-4.nii,/Users/abhinavkbij/Documents/projects/pytorchIntro/data/lits17/segmentation-5-patch-4.nii


In [90]:
patch_image.shape, patch_label.shape

((20, 224, 224), (20, 224, 224))

In [91]:
dls = SegmentationDataLoaders3D.from_df(patch_d, path = '/',
                                batch_tfms = [RandomPerspective3D(224, 0.5), 
                                              *aug_transforms_3d(p_all=0.15, noise=False)],
                                bs = 1, 
                                val_bs = 1,
                                splitter = ColSplitter('is_valid'))

/opt/miniconda3/envs/faimed3d_py310/lib/python3.10/site-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)


In [92]:
def dice(input, target):
    iflat = input.contiguous().view(-1)
    tflat = target.contiguous().view(-1)
    intersection = (iflat * tflat).sum()
    return ((2. * intersection) /
           (iflat.sum() + tflat.sum()))

def dice_score(input, target):
    return dice(input.argmax(1), target)

def dice_loss(input, target): 
    return 1 - dice(input.softmax(1)[:, 1], target)

def loss(input, target):
    return dice_loss(input, target) + nn.CrossEntropyLoss()(input, target[:, 0])

In [93]:
learn = unet_learner_3d(dls, r3d_18, n_out=2, 
                        loss_func = loss,
                        metrics = dice_score,
                        model_dir = REPO_DIR/'models',
                        cbs = [SaveModelCallback(monitor='dice_score')]
                       )
learn = learn.to_fp16()

/opt/miniconda3/envs/faimed3d_py310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/miniconda3/envs/faimed3d_py310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /Users/abhinavkbij/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth


100%|██████████| 127M/127M [00:58<00:00, 2.28MB/s] 


In [94]:
learn.lr_find()

/opt/miniconda3/envs/faimed3d_py310/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/miniconda3/envs/faimed3d_py310/lib/python3.10/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/opt/miniconda3/envs/faimed3d_py310/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/miniconda3/envs/faimed3d_py310/lib/python3.10/site-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not a

/opt/miniconda3/envs/faimed3d_py310/lib/python3.10/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:4324.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


KeyboardInterrupt: 